In [ ]:
import cv2
import os
import time

#TODO 1: Agregar la ruta del video a capturar
video_path = 'C:/git/IAClass/assets/projectU3/911_gt3_rs_red/v2.mp4'
output_directory = 'C:/git/IAClass/16_projectU3_cnn_cars/captures/911_gt3_rs_red/911_gt3_rs_red_v2'

# Init video capture
cap = cv2.VideoCapture(video_path)
frame_count = 0 
capture_count = 0

# TODO 2: Definir los rangos de color HSV para cada color
lower_red = (0, 69, 1)
upper_red = (179, 255, 255)

lower_black = (0, 0, 0)
upper_black = (178, 120, 118)

while True:
    # Capturar frame y condición de parado
    ret, frame = cap.read()
    if not ret:
        print('Video ended')
        break
    
    # Frame to hsv moden a create mask
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # TODO 3: Sumar las diferentes mascaras si es que tienen
    mask1 = cv2.inRange(hsv, lower_red, upper_red)
    mask2 = cv2.inRange(hsv, lower_black, upper_black)
    # mask3 = cv2.inRange(hsv, lower_white, upper_white)
    # mask4 = cv2.inRange(hsv, lower_shadows, upper_shadows)
    # mask5 = cv2.inRange(hsv, lower_orange_dark, upper_orange_dark)
    mask_mix_1_2 = cv2.bitwise_or(mask1, mask2)
    # mask_mix_1_2_3= cv2.bitwise_or(mask_mix_1_2, mask3)
    # mask_mix_1_2_3_4= cv2.bitwise_or(mask_mix_1_2_3, mask4)
    # mask_mix_1_2_3_4_5= cv2.bitwise_or(mask_mix_1_2_3_4, mask5)
    
    # TODO 4: Especificar la mascara final a utilizar 
    # TODO 4 (Ya sea una combinación de varias máscaras o una sola máscara)
    mask = mask_mix_1_2
    
    # Filtrar la máscara con operaciones morfológicas
    mask = cv2.erode(mask, None, iterations=0)
    mask = cv2.dilate(mask, None, iterations=0)
    
    # Encontrar contornos en la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        
        # Encontrar contorno mayor y obtener centro
        largest_contour = max(contours, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(largest_contour)
        
        # Dibujar el círculo y el centro en el frame original si el radio es mayor que un umbral
        if radius > 10:
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 255), -1)
            
            # Calcular (Esq superior izquierda y Esquina inferior derecha) 
            # y dibujar rectangulo a partir del centro y el radio
            top_left = list((int(x - radius), int(y - (radius  - 300))))
            bottom_right = list((int(x + radius), int(y + (radius - 200))))          
            cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 1)
            
            # Tomar y guardar captura cada "X" frames a partir de la región del rectángulo
            if frame_count % 1 == 0:
                
                # Normaliza recortes que salen de la pantalla
                if(top_left[0] < 0):
                    top_left[0] = 0 # Coordenada x
                if(top_left[1] < 0):
                    top_left[1] = 0 # Coordenada y
                
                cropped_image = frame[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
                
                if cropped_image.size > 0:
                    
                    capture_count += 1
                    
                    # Calcular el nuevo tamaño manteniendo la proporción
                    # target_width = 100
                    # (h, w) = cropped_image.shape[:2]
                    # aspect_ratio = w / h
                    # target_height = int(target_width / aspect_ratio)
                    resized_image = cv2.resize(cropped_image, (400, 400))
                    
                    filename = os.path.join(output_directory, f"capture_Num{capture_count}_frame_Num{frame_count}_{time.time()}.png")
                    cv2.imwrite(filename, resized_image)
                    print(f"Imagen guardada: {filename}")
                    print(f"x: {x}, y: {y}, radius: {radius}")
                    print(f"top_left: {top_left}, bottom_right: {bottom_right}")
                    
            frame_count += 1  
                  
                    
    # Mostrar el frame
    cv2.imshow('Frame', frame)
    cv2.imshow('Mask', mask)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()



Imagen guardada: C:/git/IAClass/16_projectU3_cnn_cars/captures/911_gt3_rs_red/911_gt3_rs_red_v2\capture_Num1_frame_Num0_1732423945.4084568.png
x: 636.38623046875, y: 446.0, radius: 692.4713745117188
top_left: [0, 53], bottom_right: [1328, 1438]
Imagen guardada: C:/git/IAClass/16_projectU3_cnn_cars/captures/911_gt3_rs_red/911_gt3_rs_red_v2\capture_Num2_frame_Num1_1732423945.5165224.png
x: 637.2251586914062, y: 448.99993896484375, radius: 692.0665283203125
top_left: [0, 56], bottom_right: [1329, 1441]
Imagen guardada: C:/git/IAClass/16_projectU3_cnn_cars/captures/911_gt3_rs_red/911_gt3_rs_red_v2\capture_Num3_frame_Num2_1732423945.5544896.png
x: 637.4398193359375, y: 450.5, radius: 691.6804809570312
top_left: [0, 58], bottom_right: [1329, 1442]
Imagen guardada: C:/git/IAClass/16_projectU3_cnn_cars/captures/911_gt3_rs_red/911_gt3_rs_red_v2\capture_Num4_frame_Num3_1732423945.575766.png
x: 637.4476318359375, y: 451.49993896484375, radius: 691.3001098632812
top_left: [0, 60], bottom_right: [1